In [1]:
import pandas
import warnings

from operator import eq

warnings.filterwarnings('ignore')
warnings.simplefilter  ('ignore')

pandas.set_option('display.max_rows',     500)
pandas.set_option('display.max_columns',  500)
pandas.set_option('display.width',       1000)
pandas.set_option('display.max_colwidth', 100)

from gistools.utils import read_pickle
from gistools.gmapsro import FleetRouting

In [2]:
PLATFORM = 'PF ROUEN'
output_filename = 'dataset_{}.pickle'.format(PLATFORM.replace(' ', '_'))
output_filename  

'dataset_PF_ROUEN.pickle'

In [3]:
dataset = read_pickle('_dataset/{}'.format(output_filename))

In [4]:
dataset['stores']['store_id'].unique()

array([961424, 961425, 961411, 961434, 961032, 961418, 961025, 961502],
      dtype=int64)

In [5]:
dataset['shipments'].head()

,id,shipment_name,shipment_id,input_address,input_postal_code,input_city,location_code,no_items,no_items_primary,no_items_second,no_references,weight_kg,pickup_location_id,store_pickup,setup,encasement,smart_tv,product_category_id,product_category_id_primary,product_category_name,depot_id,depot_short_name,depot_name,time_slot,duration,duration_text,duration_bin,duration_bin_text,initial_route_id,initial_route_short_name,initial_route_name,initial_route_external_id,initial_route_date,initial_route_datetime,year,month,day,weekday,eligible_team,assigned_team,initial_route_teaming,initial_route_start_time,initial_route_start_datetime,initial_route_time,initial_route_order_id,initial_route_distance,driver1_id,driver1_type,driver2_id,driver2_type,driver3_id,driver3_type,inconsistent_date,initial_route_start_time_missing,time_slot_missing,address_missing,code_99_q10,store_not_found,time_window_violation,initial_delay,initial_delay_text,customer_service,visit_type,visit_requirements,period,time_window_start,soft_time_window_start,time_window_end,soft_time_window_end,cost_per_hour_before_soft_start_time,cost_per_hour_after_soft_end_time,formatted_address,place_id,plus_code,address,postal_code,city,location_type,confidence,maps_URL,longitude,latitude,geometry,start_datetime,end_datetime,start_timestamp,end_timestamp,shipment_type
0,33675,33675,11349,340 RUE JULES FERRY,76480,DUCLAIR,411,1,1,0,1,27800,961411,False,N,N,N,TV07,['TV07'],TV LCD 60A69 POUCES,961079,PF GRAND QUEVILLY,PF ROUEN,AM,7,00:07:00,10,<=10,2023-03-01 ROUEN AA079AA,1,AA079AA,533,2023-03-01,2023-03-01 09:08:55,2023,3,1,3,2.0,Duo,duo,7:42:18,2023-03-01 07:42:18,9:08:55,1,207,AA079AA,SST,AA079AW,SST,NaN,NaN,False,False,False,False,False,False,False,0,00:00:00,SD,duo,duo,2023-03-01,07:30:00,08:00:00,14:00:00,13:00:00,10,100,"340 Rue Jules Ferry, 76480 Duclair, France",EiozNDAgUnVlIEp1bGVzIEZlcnJ5LCA3NjQ4MCBEdWNsYWlyLCBGcmFuY2UiMRIvChQKEgkht22b1frgRxGjaJzyTAGKfxDU...,8FX2FVMH+XH,340 Rue Jules Ferry,76480,Duclair,RANGE_INTERPOLATED,1.00,https://www.google.com/maps/search/?api=1&query=49.484888%2C0.878957&query_place_id=EiozNDAgUnVl...,0.878957,49.484888,POINT (0.87896 49.48489),2023-03-01 07:30:00,2023-03-01 14:00:00,1677652200,1677675600,delivery
1,33674,33674,11345,340 RUE JULES FERRY,76480,DUCLAIR,411,3,3,0,3,171100,961411,False,N,N,N,FOUR|AMERI|MLV,"['FOUR', 'AMERI', 'MLV']",FOUR A ENCASTRER|REF-AMERICAIN|LAVE VAISSELLE,961079,PF GRAND QUEVILLY,PF ROUEN,AM,21,00:21:00,30,<=30,2023-03-01 ROUEN AA079AA,1,AA079AA,533,2023-03-01,2023-03-01 09:10:16,2023,3,1,3,2.0,Duo,duo,7:42:18,2023-03-01 07:42:18,9:10:16,2,207,AA079AA,SST,AA079AW,SST,NaN,NaN,False,False,False,False,False,False,False,0,00:00:00,SD,duo,duo,2023-03-01,07:30:00,08:00:00,14:00:00,13:00:00,10,100,"340 Rue Jules Ferry, 76480 Duclair, France",EiozNDAgUnVlIEp1bGVzIEZlcnJ5LCA3NjQ4MCBEdWNsYWlyLCBGcmFuY2UiMRIvChQKEgkht22b1frgRxGjaJzyTAGKfxDU...,8FX2FVMH+XH,340 Rue Jules Ferry,76480,Duclair,RANGE_INTERPOLATED,1.00,https://www.google.com/maps/search/?api=1&query=49.484888%2C0.878957&query_place_id=EiozNDAgUnVl...,0.878957,49.484888,POINT (0.87896 49.48489),2023-03-01 07:30:00,2023-03-01 14:00:00,1677652200,1677675600,delivery
2,33673,33673,7638,3 RUE EDOUARD BRANLY,76580,LE TRAIT,122,1,1,0,1,50000,961122,False,O,N,N,CTM,['CTM'],LIVRAISON SAV,961079,PF GRAND QUEVILLY,PF ROUEN,AM,20,00:20:00,20,<=20,2023-03-01 ROUEN AA079AA,1,AA079AA,533,2023-03-01,2023-03-01 10:06:53,2023,3,1,3,5.0,Duo,mono,7:42:18,2023-03-01 07:42:18,10:06:53,3,207,AA079AA,SST,AA079AW,SST,NaN,NaN,False,False,False,False,False,False,False,0,00:00:00,Premium,mono,mono|duo,2023-03-01,07:30:00,08:00:00,14:00:00,13:00:00,10,100,"3 Rue Edouard Branly, 76580 Le Trait, France",ChIJuaJdm7f74EcRvblmILm2xAo,8FX2FRF8+XQ,3 Rue Edouard Branly,76580,Le Trait,ROOFTOP,1.00,https://www.google.com/maps/search/?api=1&query=49.474950%2C0.816934&query_place_id=ChIJuaJdm7f7...,0.816934,49.474950,POINT (0.81693 49.47495),2023-03-01 07:30:00,2023-03-01 14:00:00,1677652200,167767

In [6]:
df = dataset['shipments'].where([('store_pickup', '==', True), ('pickup_location_id', '~isin', list(dataset['stores']['store_id'].unique()))])
df

,id,shipment_name,shipment_id,input_address,input_postal_code,input_city,location_code,no_items,no_items_primary,no_items_second,no_references,weight_kg,pickup_location_id,store_pickup,setup,encasement,smart_tv,product_category_id,product_category_id_primary,product_category_name,depot_id,depot_short_name,depot_name,time_slot,duration,duration_text,duration_bin,duration_bin_text,initial_route_id,initial_route_short_name,initial_route_name,initial_route_external_id,initial_route_date,initial_route_datetime,year,month,day,weekday,eligible_team,assigned_team,initial_route_teaming,initial_route_start_time,initial_route_start_datetime,initial_route_time,initial_route_order_id,initial_route_distance,driver1_id,driver1_type,driver2_id,driver2_type,driver3_id,driver3_type,inconsistent_date,initial_route_start_time_missing,time_slot_missing,address_missing,code_99_q10,store_not_found,time_window_violation,initial_delay,initial_delay_text,customer_service,visit_type,visit_requirements,period,time_window_start,soft_time_window_start,time_window_end,soft_time_window_end,cost_per_hour_before_soft_start_time,cost_per_hour_after_soft_end_time,formatted_address,place_id,plus_code,address,postal_code,city,location_type,confidence,maps_URL,longitude,latitude,geometry,start_datetime,end_datetime,start_timestamp,end_timestamp,shipment_type
69,33734,33734,652,1927 ROUT DE CONDE,27500,PONT AUDEMER,260,1,0,1,1,50000,961260,True,O,N,N,LVTNO,[],LIVRAISON TIERS NORD,961079,PF GRAND QUEVILLY,PF ROUEN,AM,7,00:07:00,10,<=10,2023-03-01 ROUEN AA079EE,7,AA079EE,538,2023-03-01,2023-03-01 11:42:26,2023,3,1,3,2.0,Duo,duo,7:49:22,2023-03-01 07:49:22,11:42:26,5,207,AA079EE,SST,AA079FF,SST,NaN,NaN,False,False,False,False,False,True,False,0,00:00:00,Premium,duo,duo,2023-03-01,07:30:00,08:00:00,14:00:00,13:00:00,10,100,"1927 Rte de Condé, 27500 Pont-Audemer, France",ChIJOVr_CfCs4UcRDvGVqwi33O4,8FX28HJH+XJ,1927 Route de Condé,27500,Pont-Audemer,ROOFTOP,0.97,https://www.google.com/maps/search/?api=1&query=49.332494%2C0.579000&query_place_id=ChIJOVr_CfCs...,0.579000,49.332494,POINT (0.57900 49.33249),2023-03-01 07:30:00,2023-03-01 14:00:00,1677652200,1677675600,store_pickup_and_delivery
232,33823,33823,673,9 RUE DU BOSC CARRE,27260,EPAIGNES,260,1,0,1,1,40000,961260,True,O,N,N,LVTNO,[],LIVRAISON TIERS NORD,961079,PF GRAND QUEVILLY,PF ROUEN,AM,7,00:07:00,10,<=10,2023-03-02 ROUEN AA079EE,7,AA079EE,535,2023-03-02,2023-03-02 10:41:47,2023,3,2,4,2.0,Duo,duo,7:52:43,2023-03-02 07:52:43,10:41:47,4,267,AA079EE,SST,AA079FF,SST,NaN,NaN,False,False,False,False,False,True,False,0,00:00:00,Premium,duo,duo,2023-03-02,07:30:00,08:00:00,14:00:00,13:00:00,10,100,"9 Rue du Bosc Carré, 27260 Épaignes, France",ChIJ-WBqF1ix4UcRyecqTtjsHNE,8FX27CJW+VW,9 Rue du Bosc Carré,27260,Épaignes,ROOFTOP,1.00,https://www.google.com/maps/search/?api=1&query=49.282233%2C0.447352&query_place_id=ChIJ-WBqF1ix...,0.447352,49.282233,POINT (0.44735 49.28223),2023-03-02 07:30:00,2023-03-02 14:00:00,1677738600,1677762000,store_pickup_and_delivery
809,34469,34469,678,248 RUE DE LA CABOTERIE,27500,TOCQUEVILLE,260,1,0,1,1,40000,961260,True,O,N,N,LVTNO,[],LIVRAISON TIERS NORD,961079,PF GRAND QUEVILLY,PF ROUEN,PM,7,00:07:00,10,<=10,2023-03-08 ROUEN AA079EE,9,AA079EE,538,2023-03-08,2023-03-08 14:18:37,2023,3,8,3,2.0,Duo,duo,7:54:14,2023-03-08 07:54:14,14:18:37,10,216,AA079EE,SST,AA079FF,SST,NaN,NaN,False,False,False,False,False,True,False,0,00:00:00,Premium,duo,duo,2023-03-08,11:45:00,12:00:00,17:15:00,17:00:00,10,10,"248 La Caboterie, 27500 Tocqueville, France",ChIJ884X0n5U4EcRo5uDExQRXMA,8FX2CJ58+JR,248 La Caboterie,27500,Tocqueville,ROOFTOP,0.91,https://www.google.com/maps/search/?api=1&query=49.409124%2C0.617008&query_place_id=ChIJ884X0n5U...,0.617008,49.409124,POINT (0.61701 49.40912),2023-03-08 11:45:00,2023-03-08 17:15:00,1678272300,1678292100,store_pickup_and_delivery


In [7]:
dataset['shipments'].loc[df.index, 'store_pickup']  = False 
dataset['shipments'].loc[df.index, 'shipment_type'] = 'delivery'

In [8]:
problem = FleetRouting(
    data=dataset, 
    expr=('period', '==', '2023-03-01'),
    populate_polylines=True,
    utc_offset=1,
    label='Test'
).describe()


Summary:
   169 shipments
     1 depots
    16 vehicles


In [9]:
problem.build_scenario().save_scenario(filename='scenario.json')

<FleetRouting>

In [10]:
problem.load_solution(filename='scenario.zip').describe()


Summary:
   169 shipments
     1 depots
    16 vehicles

KPIs:
   Number of routes: 13     
   Skipped visits:   0     
   Productivity:     13.2   
   Distance:         2786.972 km
   Average distance: 214.382 km
   Service time:     48 % 
   Travel time:      52 % 
   Total cost:       5105.4 €


<FleetRouting>

In [ ]:
problem.plot_solution()